# Parallel job execution with joblib

This notebook contains an example showing how to execute a function on multiple remote workers in parallel.

In [1]:
import numpy as np

import dask,time
import dask.bag
from sklearn.externals import joblib

In [2]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster.from_yaml('../worker.yaml')
cluster

In [3]:
from dask.distributed import Client, progress
c = Client(cluster)
c

Client Scheduler: tcp://10.16.1.11:36925 Dashboard: /user/ingmar/proxy/44900/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
c.get_versions(check=True)

{'scheduler': {'host': (('python', '3.6.3.final.0'),
   ('python-bits', 64),
   ('OS', 'Linux'),
   ('OS-release', '4.4.111+'),
   ('machine', 'x86_64'),
   ('processor', 'x86_64'),
   ('byteorder', 'little'),
   ('LC_ALL', 'en_US.UTF-8'),
   ('LANG', 'en_US.UTF-8'),
   ('LOCALE', 'en_US.UTF-8')),
  'packages': {'required': (('dask', '0.18.2'),
    ('distributed', '1.22.1'),
    ('msgpack', '0.5.6'),
    ('cloudpickle', '0.6.1'),
    ('tornado', '5.0.2'),
    ('toolz', '0.9.0')),
   'optional': (('numpy', '1.15.1'),
    ('pandas', '0.23.2'),
    ('bokeh', '0.12.16'),
    ('lz4', '2.1.1'),
    ('blosc', '1.5.1'))}},
 'workers': {'tcp://10.16.22.8:33176': {'host': (('python', '3.6.3.final.0'),
    ('python-bits', 64),
    ('OS', 'Linux'),
    ('OS-release', '4.4.111+'),
    ('machine', 'x86_64'),
    ('processor', 'x86_64'),
    ('byteorder', 'little'),
    ('LC_ALL', 'en_US.UTF-8'),
    ('LANG', 'en_US.UTF-8'),
    ('LOCALE', 'en_US.UTF-8')),
   'packages': {'required': (('dask', '0.18.

In [5]:
def very_hard_problem(zahl):
    res = 0
    for n in np.arange(zahl):
        res = res + n
    return res

In [6]:
tic = time.time()
very_hard_problem(10000000)
print(time.time() - tic,'seconds')

1.1631879806518555 seconds


In [14]:
num_procs = 20

In [15]:
tic = time.time()
with joblib.parallel_backend('dask'):
    result_list = joblib.Parallel()(
        joblib.delayed(very_hard_problem)(i)
        for i in np.arange(10000000,10000000 + num_procs)
    )
time_elapsed = time.time() - tic

print(str(time_elapsed) + ' seconds in total, ' + str(time_elapsed/num_procs) + ' per process')

2.54291033744812 seconds in total, 0.127145516872406 per process


In [16]:
params_bag = dask.bag.from_sequence(np.arange(10000000,10000000 + num_procs))

In [17]:
tic = time.time()
result_list = params_bag.map(lambda x: very_hard_problem(x)).compute()
time_elapsed = time.time() - tic

print(str(time_elapsed) + ' seconds in total, ' + str(time_elapsed/num_procs) + ' per process')

2.473707675933838 seconds in total, 0.12368538379669189 per process
